In [ ]:
!pip install transformers huggingface datasets langchain langchain-chroma

  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 14.9 MB/s eta 0:00:00


In [4]:
!pip install torch

  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 k

In [2]:
!pip install ipywidgets langchain-community

  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)


In [6]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.6 MB/s eta 0:00:00a 0:00:01


In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
from tqdm.notebook import tqdm  # For progress bar

In [1]:
from datasets import load_dataset
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm.notebook import tqdm

# Load dataset
dataset = load_dataset("Vivacem/MMIQC", cache_dir='/data/user_data/amittur')
questions = dataset['train']['instruction']
answers = dataset['train']['output']

# Initialize embedding model
embedding_model_name = "math-similarity/Bert-MLM_arXiv-MP-class_zbMath"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"trust_remote_code": True}
)

# Initialize Chroma
persist_directory = "/data/user_data/amittur/MMIQC_ChromaDB"
vectorstore = Chroma(
    embedding_function=embeddings_model,
    persist_directory=persist_directory,
)

# # Prepare documents
documents = [{"id": str(idx), "content": q, "metadata": {"answer": answers[idx]}} 
            for idx, q in enumerate(questions)]

# # Process in batches
BATCH_SIZE = 1024 
# for i in tqdm(range(0, len(documents), BATCH_SIZE), desc="Processing Batches"):
#     batch = documents[i:i + BATCH_SIZE]
#     vectorstore.add_texts(
#         texts=[doc["content"] for doc in batch],
#         metadatas=[doc["metadata"] for doc in batch],
#         ids=[doc["id"] for doc in batch]
#     )
batch = documents
vectorstore.add_texts(
    texts=[doc["content"] for doc in batch],
    metadatas=[doc["metadata"] for doc in batch],
    ids=[doc["id"] for doc in batch]
)

# # Persist to disk
vectorstore.persist()
print("Vector embeddings created and stored successfully!")

/tmp/ipykernel_2433460/685918297.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/tmp/ipykernel_2433460/685918297.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


: 

In [3]:
# Step 4: Query the vector store to retrieve top-k similar questions
def get_top_k_similar_questions(query, top_k=5):
    results = vectorstore.similarity_search(query, k=top_k)
    return [
        {
            "question": result.page_content,
            "solution": result.metadata["answer"]
        }
        for result in results
    ]

In [4]:
def get_question_prompt(question):
    prefix = 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\n'
    return prefix + question

In [5]:
# Print the texts in the vecotrstore
print("Querying the vector store...")
question = '"What is the value of 2 + 2?"'
query = get_question_prompt(question)
similar_questions = get_top_k_similar_questions(query, top_k=10)
print("Top 5 similar questions:")
for idx, question in enumerate(similar_questions):
    print(f"Question {idx + 1}: {question['question']}")
    print(f"Answer: {question['solution']}\n")
    print("---------------------------------------------------")

Querying the vector store...
Top 5 similar questions:
Question 1: Please solve the following problem and put your answer at the end with "The answer is: ".

What is $\frac{2}{5}$ divided by 3?


Answer: To divide $\frac{2}{5}$ by 3, we can rewrite it as $\frac{2}{5}\div\frac{3}{1}$.
Dividing by a fraction is the same as multiplying by its reciprocal, so we have $\frac{2}{5}\cdot\frac{1}{3}$.
Multiplying the numerators and denominators, we get $\frac{2\cdot1}{5\cdot3}$.
Simplifying, we have $\frac{2}{15}$.
Therefore, $\frac{2}{5}$ divided by 3 is $\boxed{\frac{2}{15}}$.
The answer is: \frac{2}{15}

---------------------------------------------------
Question 2: Please solve the following problem and put your answer at the end with "The answer is: ".

What is $\frac{2}{5}$ divided by 3?


Answer: To divide a fraction by a whole number, you can multiply the denominator of the fraction by the whole number. Alternatively, you can think of it as multiplying the fraction by the reciprocal of 

In [6]:
similar_questions[:3]

[{'question': 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\nWhat is $\\frac{2}{5}$ divided by 3?\n\n',
  'solution': 'To divide $\\frac{2}{5}$ by 3, we can rewrite it as $\\frac{2}{5}\\div\\frac{3}{1}$.\nDividing by a fraction is the same as multiplying by its reciprocal, so we have $\\frac{2}{5}\\cdot\\frac{1}{3}$.\nMultiplying the numerators and denominators, we get $\\frac{2\\cdot1}{5\\cdot3}$.\nSimplifying, we have $\\frac{2}{15}$.\nTherefore, $\\frac{2}{5}$ divided by 3 is $\\boxed{\\frac{2}{15}}$.\nThe answer is: \\frac{2}{15}'},
 {'question': 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\nWhat is $\\frac{2}{5}$ divided by 3?\n\n',
  'solution': "To divide a fraction by a whole number, you can multiply the denominator of the fraction by the whole number. Alternatively, you can think of it as multiplying the fraction by the reciprocal of the whole number. Let's use the second method:\n\n